I load the data.
I randomly initialize the probability of picking a class and random lambdas.
Then I run the EM loop until convergence.

# Part A

The proper distribution for number of bids per day is a Poisson distribution with a single parameter called lambda. This lambda is the mean. P (X = k) = (lambda ^ k * e ^ (-lambda)) / k!

In [7]:
import os
import numpy as np
from scipy.stats import poisson
from dotenv import load_dotenv


load_dotenv()

True

In [8]:
file_path = os.environ.get("DATASET_PATH") + "pb2data_D.txt"

dataset = np.loadtxt(file_path, delimiter=",")
print("Dataset loaded successfully.")
print("Dataset shape:", dataset.shape)
print("Dataset first 5 rows:\n", dataset[:5])

Dataset loaded successfully.
Dataset shape: (10000,)
Dataset first 5 rows:
 [12. 22. 21. 12.  2.]


In [9]:
chance_of_picking_class = np.random.dirichlet([2, 5, 4, 3])
print("Chance of picking each class:", chance_of_picking_class)

k = 4
max_iter = 100
threshold = 1e-8
prev_likelihood = -np.inf
lambdas = np.random.uniform(1, 25, size=k)

responsibilities = np.zeros((dataset.shape[0], k))

for iteration in range(max_iter):
    # E-step: determine responsibilities
    for i in range(k):
        responsibilities[:, i] = chance_of_picking_class[i] * poisson.pmf(dataset, mu=lambdas[i])

    responsibilities /= responsibilities.sum(keepdims=True, axis=1)
    
    # M-step: update parameters
    Nk = responsibilities.sum(axis=0)
    chance_of_picking_class = Nk / dataset.shape[0]
    lambdas = (responsibilities * dataset[:, np.newaxis]).sum(axis=0) / Nk

    likelihood = np.sum(np.log(np.sum(chance_of_picking_class * poisson.pmf(lambdas[:, None], mu=lambdas), axis=1)))
    # Check for convergence
    if abs(likelihood - prev_likelihood) < threshold:
        print(f"Converged after {iteration} iterations.")
        break

    prev_likelihood = likelihood

Chance of picking each class: [0.10492638 0.19322896 0.49480319 0.20704147]


/var/folders/9d/hd3srjbx4rgbbrvnwxmcmh9c0000gn/T/ipykernel_40307/2041061204.py:24: RuntimeWarning: divide by zero encountered in log
  likelihood = np.sum(np.log(np.sum(chance_of_picking_class * poisson.pmf(lambdas[:, None], mu=lambdas), axis=1)))
/var/folders/9d/hd3srjbx4rgbbrvnwxmcmh9c0000gn/T/ipykernel_40307/2041061204.py:26: RuntimeWarning: invalid value encountered in scalar subtract
  if abs(likelihood - prev_likelihood) < threshold:


In [13]:
print("Final values of lambdas: ", lambdas)
print("Final chance of picking classes: ", chance_of_picking_class*10000)

Final values of lambdas:  [13.96559424  5.95768166  1.92467108 22.09204597]
Final chance of picking classes:  [1101.47208767 3077.22913692 1871.05481928 3950.24395614]
